# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents configured in Agent Lab in watsonx.ai. It introduces Python API commands for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully. If code is modified or reordered, there is no guarantee it will successfully execute. For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response 

# Setup


In [ ]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests


## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
    return {
        "url" : "https://us-south.ml.cloud.ibm.com",
        "apikey" : getpass.getpass("Please enter your api key (hit enter): ")
    }

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()


# Using the agent
These cells demonstrate how to create and invoke the agent with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [ ]:
model_id = "meta-llama/llama-3-3-70b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the result:


In [ ]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}


## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain the id from the project or space in which this notebook runs:


In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating the agent
We need to create the agent using the properties we defined so far:


In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model


In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)

def create_python_interpreter_tool(context):
    from langchain_core.tools import StructuredTool

    import ast
    import sys
    from io import StringIO
    import uuid
    import base64
    import os

    original_import = __import__

    def get_image_url(base_64_content, image_name, context):
        url = "https://api.dataplatform.cloud.ibm.com"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f'Bearer {context.get_token()}'
        }

        body = {
            "name": image_name,
            "blob": base_64_content
        }

        params = {
            "project_id": project_id
        }

        response = requests.post(f'{url}/wx/v1-beta/utility_agent_tools/resources', headers=headers, json=body, params=params)

        return response.json().get("uri")

    def patched_import(name, globals=None, locals=None, fromlist=(), level=0):
        module = original_import(name, globals, locals, fromlist, level)

        if name == "matplotlib.pyplot":
            sys.modules["matplotlib.pyplot"].show = pyplot_show
        return module

    def pyplot_show():
        pictureName = "plt-" + uuid.uuid4().hex + ".png"
        plt = sys.modules["matplotlib.pyplot"]
        plt.savefig(pictureName)
        with open(pictureName, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
            print(f"base64image:{pictureName}:{str(encoded_string)}")
            os.remove(pictureName)
            plt.clf()
            plt.close("all")

    def init_imports():
        import builtins
        builtins.__import__ = patched_import

    def _executeAgentCode(code):
        old_stdout = sys.stdout
        try:
            full_code = "init_imports()\n\n" + code
            tree = ast.parse(full_code, mode="exec")
            compiled_code = compile(tree, 'agent_code', 'exec')
            namespace = {"init_imports": init_imports}
            redirected_output = sys.stdout = StringIO("")
            exec(compiled_code, namespace)
            value = redirected_output.getvalue()
            if (value.startswith("base64image")):
                image_details = value.split(":")
                image_name = image_details[1]
                base_64_image = image_details[2]
                image_url = get_image_url(base_64_image, image_name, context)
                value = f"Result of executing generated Python code is an image:\n\nIMAGE({image_url})"
        except Exception as e:
            value = "Error while executing Python code:\n\n" + str(e)
        finally:
            sys.stdout = old_stdout
        return value

    tool_description = """Run Python code and return the console output. Use for isolated calculations, computations or data manipulation. In Python, the following modules are available: Use numpy, pandas, scipy and sympy for working with data. Use matplotlib to plot charts. Other Python libraries are also available -- however, prefer using the ones above. Prefer using qualified imports -- `import library; library.thing()` instead of `import thing from library`. Do not attempt to install libraries manually -- it will not work. Do not use this tool multiple times in a row, always write the full code you want to run in a single invocation. If you get an error running Python code, try to generate a better one that will pass. If the tool returns result that starts with IMAGE(, follow instructions for rendering images."""
    tool_schema = {
        "type": "object",
        "$schema": "http://json-schema.org/draft-07/schema#",
        "properties": {
            "code": {
                "description": "Code to be executed.",
                "type": "string"
            }
        },
        "required": ["code"]
    }

    return StructuredTool(
        name="PythonInterpreter",
        description = tool_description,
        func=_executeAgentCode,
        args_schema=tool_schema
    )


vector_index_id = "40824957-150a-4607-a08c-7f8885b0befa"

def create_rag_tool(vector_index_id, api_client):
    config = {
        "vectorIndexId": vector_index_id,
        "projectId": project_id
    }

    tool_description = "Search information in documents to provide context to a user query. Useful when asked to ground the answer in specific knowledge about Mock Bank Policy Global"

    return create_utility_agent_tool("RAGQuery", config, api_client, tool_description=tool_description)



def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")

    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }

    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )

        return results.get("output")

    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [x for x in tree.body if isinstance(x, ast.FunctionDef)]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)

    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []

    name_policy_retrieval_test_py_tfmz9 = "policy_retrieval_test.py"
    desc_policy_retrieval_test_py_tfmz9 = """Test"""
    code_policy_retrieval_test_py_tfmz9 = """import pytest
import time


@pytest.fixture(scope="session", autouse=True)
def refresh_token_before_tests():
    from main_script import get_iam_token  # or the function you defined
    global access_token
    access_token = get_iam_token("YOUR_IBM_API_KEY")
    print("
[Token refreshed before test session]")
    yield
    print("
[Tests complete; token expired naturally after session]")


import os
import json
import datetime
from textwrap import dedent

import requests


with open("evaluation_prompts.json") as f:
    eval_prompts = json.load(f)


SYSTEM_PROMPT = dedent(
    """You are "Loan Risk Assistant", a bank-compliant copilot for loan officers and borrowers.

## Mission
Given borrower/application details, produce:
- A clear risk score summary (from the Risk Scoring API, not guessed)
- Human-readable reasons (linked to policies/evidence)
- Required docs to request next
- A suggested interest-rate band (policy-constrained, not speculative)
- Citations to policy chunks and retrieved docs
- An audit payload for watsonx.governance

## Tools available
1) policy_docs_retriever(query, top_k)  → RAG over Docling→Embeddings→VectorDB
2) risk_scoring_api(payload)            → Deterministic risk model; returns {score, features, reason_codes}
3) get_policy_by_id(ids[])              → Fetch exact policy text for cited chunk_ids
4) compose_user_packet(data)            → Format final packet for user delivery (HTML/text)
5) request_additional_docs(list)        → Generate borrower request list in CRM
6) governance_log(event_type, payload)  → Log every action (inputs, outputs, tool calls)

## Ground rules
- **Do not fabricate scores, policy text, or citations.** Always call the risk API for scores. Always cite chunk_ids returned by retrieval.
- Prefer **bank policy** over model in case of conflict; call get_policy_by_id to confirm wording before quoting.
- **Never reveal internal embeddings, prompt tokens, or staff-only notes** to end users.
- **No personal advice** beyond bank policy; use neutral language.
- If inputs are incomplete, **ask for the minimum next docs** required by policy, not everything.
- Interest suggestion = **policy-bounded band**. If policy missing, return "interest_rate_suggestion": null and add a **policy_gap flag**.
- Provide **short, skimmable** reasons with numbered bullets; each bullet must map to a reason_code or retrieved policy chunk.
- Include **region + product** constraints when retrieving policies (e.g., state, product = {auto, mortgage, SMB term}, risk tiering rules).
- **Log** each tool action with governance_log before returning the final response.

## Required output format (JSON)
Return a single JSON object with:
{
  "application_id": "<string>",
  "risk_score": { "value": <number>, "scale": "0-100", "tier": "<Low|Med|High>" },
  "reasons": [
    { "label": "<short>", "detail": "<1-2 sentences>", "source": {"type": "policy|feature", "id_or_code": "<chunk_id|reason_code>"} }
  ],
  "policy_citations": [
    { "chunk_id": "<id>", "title": "<doc title>", "section": "<sec>", "quote": "<<=50 words exact>>" }
  ],
  "requested_documents": ["<doc 1>", "<doc 2>", "..."],
  "interest_rate_suggestion": { "band_apr_percent": [min, max], "basis": "<policy_ref>", "conditions": ["<e.g., auto-pay>", "..."] } | null,
  "compliance": {
    "region": "<state/country>",
    "product": "<loan product>",
    "policy_gap": false
  },
  "governance_log_ids": ["<id1>", "<id2>", "..."],
  "user_packet": { "format": "html", "content": "<rendered summary for user>" }
}

## Workflow
1) Governance log: problem_received(application_id, redactions=true)
2) Retrieve policies: policy_docs_retriever with product+region+keywords derived from the user input; cite chunk_ids.
3) Score: risk_scoring_api with structured payload (income, DTI, FICO, LTV, collateral, delinq, purpose, term, region, product).
4) Validate & enrich reasons: map reason_codes → plain language; cross-check against retrieved policy chunks; fetch exact quotes via get_policy_by_id for any cited chunk_ids you plan to quote.
5) Decide **requested_documents**: enforce minimum-necessary principle from policy (e.g., proof of income variant depends on employment type).
6) Interest band: derive from policy tables/logic; if insufficient policy evidence, set to null and set policy_gap=true.
7) Compose user-facing packet; keep internal IDs out of user view. Write neutral, actionable language.
8) Governance log each step (retrieval_done, risk_scored, packet_composed) with input/output hashes.
9) Return the JSON object exactly once.

## Style
- Executive-brief length. 5–8 bullets max in reasons.
- Cite policy chunks minimally (2–5), but precisely (section + <=50 words).
- No speculative language. If uncertain, state what is missing and request docs.

Proceed when the user provides or updates application data."""
)


def test_policy_retrieval_api():
    access_token = os.getenv("ACCESS_TOKEN")
    if not access_token:
        raise RuntimeError("ACCESS_TOKEN environment variable must be set for this test")

    url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/chat?version=2023-05-29"

    model_id = "ibm/granite-3-3-8b-instruct"

    for case in eval_prompts:
        prompt_input = case["prompt"]

        body = {
            "messages": [
                {"role": "system", "content": SYSTEM_PROMPT},
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt_input}
                    ],
                },
            ],
            "project_id": "da24b80f-68d9-4b42-8c83-aaf5da05dcba",
            "model_id": model_id,
            "frequency_penalty": 0,
            "max_tokens": 2000,
            "presence_penalty": 0,
            "temperature": 0,
            "top_p": 1,
        }

        response = requests.post(
            url,
            headers={
                "Accept": "application/json",
                "Content-Type": "application/json",
                "Authorization": f"Bearer {access_token}",
            },
            json=body,
            timeout=30,
        )

        if response.status_code != 200:
            raise AssertionError(f"Non-200 response: {response.text}")

        data = response.json()
        model_response = data

        log_entry = {
            "timestamp": datetime.datetime.utcnow().isoformat(),
            "prompt": prompt_input,
            "response": model_response,
            "model": model_id,
            "retrieval_index": "Mock Bank Policy Global",
        }

        with open("policy_retrieval_logs.json", "a") as f:
            f.write(json.dumps(log_entry) + "
")

        print(f"Prompt: {prompt_input}")
        print(f"Response: {model_response}
")

        assert isinstance(data, dict)
"""
    params_policy_retrieval_test_py_tfmz9 = {
    }

    schema_policy_retrieval_test_py_tfmz9 = {
        "type": "object",
        "properties": {
            "prompt": {
                "type": "string",
                "title": "Policy query",
                "description": "The natural-language question to ask the policy retrieval model"
            }
        }
    }
    custom_tools.append(create_custom_tool(name_policy_retrieval_test_py_tfmz9, desc_policy_retrieval_test_py_tfmz9, code_policy_retrieval_test_py_tfmz9, schema_policy_retrieval_test_py_tfmz9, params_policy_retrieval_test_py_tfmz9))

    return custom_tools


def create_tools(context):
    tools = []
    tools.append(create_rag_tool(vector_index_id, client))
    tools.append(create_python_interpreter_tool(context))

    config = None
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))

    tools = tools + create_custom_tools()
    return tools

def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """# Notes
- When a tool is required to answer the user's query, respond only with <|tool_call|> followed by a JSON list of tools used.
- If a tool does not exist in the provided list of tools, notify the user that you do not have the ability to fulfill the request.
You are Loan Risk Assistant, an enterprise AI agent for evaluating loan applications
according to documented bank policies.

## Purpose
Assess risk, cite relevant policy rules, and produce a JSON report suitable for
watsonx.governance logging.

## Responsibilities
1. Compute a normalized risk score (0–1) with tier label (Low, Medium, High).
2. Explain each reason concisely and cite the source policy section or reason code.
3. Identify any required supporting documents.
4. Suggest an interest-rate band only if the cited policy defines one.
5. Always include compliance metadata (region, product, policy_gap flag).
Loan Risk Tier Documentation Requirements
1. LOW-RISK APPLICANTS (Low Tier)

Required Documents:

Valid Government-issued ID (for identity verification)

Passport / Driver’s License / National ID

Proof of Income (to confirm repayment ability)

Latest 3 months’ payslips

Certificate of Employment (COE)

Bank statement showing salary credits (3 months)

Proof of Address (for KYC and correspondence)

Utility bill or lease contract

Credit Report or consent to pull one

Loan Application Form (digitally filled and signed)

Collateral Documents (if applicable)

Vehicle OR/CR, land title, or other asset proof

System Behavior / Decision Logic:

Risk Output: Accepted / Auto-approved (conditional)

Action: Proceed to underwriting or direct approval if within policy thresholds

Audit Log: System auto-logs approval with justification referencing risk score and compliance rules.

2. HIGH-RISK APPLICANTS (High Tier)

Required Documents (Expanded Set):

Valid Government-issued ID

Proof of Income (Comprehensive):

Latest 6 months of payslips or income statements

Income Tax Return (ITR) or Audited Financial Statement (for business owners/self-employed)

Business Permit / DTI Certificate / SEC Registration (if applicable)

Proof of Address

Collateral Documents (Mandatory for high-tier applicants):

Real estate titles, vehicle ownership papers, or deposit certificates as security

Statement of Assets and Liabilities (SALN or equivalent)

Bank Statements (last 6 months) – for cash flow verification

Guarantor or Co-maker Agreement (if required by policy)

Supporting Proof of Loan Purpose:

Purchase order, contract, or business plan for loan justification

Proof of Capacity / Means to Pay:

Additional income sources (e.g., remittances, side business receipts, investment returns)

Letter of Explanation (optional, to justify financial stability)

System Behavior / Decision Logic:

Risk Output: Denied (Flagged for manual review)

Action: Temporarily marked as “Denied – Awaiting Supporting Documents”

Audit Log: System records reason for denial and triggers a request for additional documentation.

Once additional proof is submitted and verified, the system can recalculate risk score or forward to manual credit officer review.

f risk_tier == "high":
    status = "Denied (Pending Additional Proof)"
    required_docs = ["ID", "Proof of Income (6 months)", "Collateral Docs", "ITR/Audited FS", "Bank Statements", "Guarantor Agreement"]
else:
    status = "Accepted"
    required_docs = ["ID", "Proof of Income (3 months)", "Proof of Address"]

governance_log("High-risk applicant flagged. Requesting additional proof of capacity.")

## OUTPUT
Output and then summarize the JSON in the chat while also giving approval or disapproval based on policy

## Constraints
- Never fabricate data, scores, or policy text.
- Prefer policy evidence over model inference.
- Use neutral, factual tone suitable for internal audit.
- If data are incomplete, state what is missing and request only the minimum
  additional documents.
- Do not include user PII in outputs.
- Log every inference using the governance client after completion.

Be precise, transparent, and auditable.
"""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent


In [ ]:
# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")


# Next steps
You successfully completed this notebook! You learned how to use watsonx.ai inferencing SDK to generate response from the foundation model based on the provided input, model id and model parameters. Check out the official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License. Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>
